In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

First of all, the libraries and data need to be imported:

In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import plotly.offline as py
import plotly.graph_objs as go
import matplotlib.ticker as mtick  
import matplotlib.pyplot as plt
%matplotlib inline


import os
print("Input data file:  ",os.listdir("../input/pima-indians-diabetes-database"))

sns.set(style = 'white')



In [ ]:
diabetes = pd.read_csv('../input/pima-indians-diabetes-database/diabetes.csv')

As a first step, some basic EDA is done to get an overview of the data and have an idea what it contains, and what it looks like.
To show the column names:

In [ ]:
print("\n".join(diabetes.columns.values))

To show the first rows of the data:

From the zero values in the columns Skin Thickness and Insulin, it already seems the dataset is not complete and the missing values are represented by 0:

In [ ]:
diabetes.head(20)

To show the last rows of the dataset:

In [ ]:
diabetes.tail()

To show the number of rows and columns in the dataset:

In [ ]:
diabetes.shape

To show the types of data:

In [ ]:
diabetes.dtypes

To show basic statistical details:

In [ ]:
diabetes.describe()

To make a copy of the original data, so that any further analysis is made on the copy and the original data stays safe and unchanged:

In [ ]:
diabetes_data = diabetes.copy()

To see a summary of the data, like column data types or non-null values.

It can be seen that there are no real null values in the data, but as the head and tail functions suggested, null values are represented by zero.


In [ ]:
diabetes_data.info()

In [ ]:
diabetes_data.isnull().sum()

Therefore the attempt to show null values in Blood Pressure brings no result:

In [ ]:
diabetes_data.loc[diabetes_data ['BloodPressure'].isnull() == True]

To show missing values in Blood pressure, rows with value 0 need to be found:

In [ ]:
diabetes_data[diabetes_data["BloodPressure"] == 0]

To impute this mising data in blood pressure and BMI, the mean is used:

In [ ]:
mean_value_bloodpressure = diabetes_data['BloodPressure'].mean()
diabetes_data.loc[diabetes_data ['BloodPressure'] == 0 ,'BloodPressure'] = mean_value_bloodpressure
diabetes_data.loc[diabetes_data ['BloodPressure'] == mean_value_bloodpressure]

In [ ]:
mean_value_BMI = diabetes_data['BMI'].mean()
diabetes_data.loc[diabetes_data ['BMI'] == 0 ,'BMI'] = mean_value_BMI
diabetes_data.loc[diabetes_data ['BMI'] == mean_value_BMI]

To show how the BMI values are spread in the dataset - the most frequest BMI value is around 30, therefore the majority of the people in the survey are considered obese.

In [ ]:
diabetes_data['BMI'].plot(kind='hist')

The data can be split into bins based on the BMI values according to CDC rates, here I tried different representations:

In [ ]:
diabetes_data['bin'] = pd.cut(diabetes_data['BMI'], [0, 18.5, 25, 30, 100], labels=['0-18.5', '18.5-25', '25-30', '30-100'])
print(diabetes_data['BMI'], diabetes_data['bin'])

In [ ]:
bins = [-np.inf, 18.5, 25, 30, 100, np.inf]
diabetes_data['BMIBins'] = pd.cut(diabetes_data['BMI'], bins)
diabetes_data

In [ ]:
bins = [0, 18.5, 25, 30, 100]
diabetes_data['BMIBins'] = pd.cut(diabetes_data['BMI'], bins)
diabetes_data

In [ ]:
bins = [0, 18.5, 25, 30, 100]
names = ['Underweight', 'Normal', 'Overweight', 'Obese']

diabetes_data['BMIRange'] = pd.cut(diabetes_data['BMI'], bins, labels=names)
print(diabetes_data['BMI'], diabetes_data['BMIRange'])

To show the data types also of the newly created variables:

In [ ]:
diabetes_data.dtypes

In [ ]:
print(diabetes_data['BMIBins'].value_counts())

In [ ]:
print(diabetes_data['BMIRange'].value_counts())

To show the correlation of the the individual dependent variables with relation to the independent variable Outcome  = whether a person has diabetes or not.
The highest correlation is with Glucose, so it can be considered the most important predictor.

In [ ]:
Corr_Outcome = diabetes_data.corr()["Outcome"]
plt.figure(figsize=(10,5))
Corr_Outcome.drop('Outcome').sort_values(ascending = False).plot(kind='bar')

There were missing values also in the columns Insulin, Glucose and Skin Thickness, these are also replaced by their mean value below:

In [ ]:
def median_target(var):   
    temp = diabetes_data[diabetes_data[var].notnull()]
    temp = temp[[var, 'Outcome']].groupby(['Outcome'])[[var]].median().reset_index()
    return temp

In [ ]:
median_target('Insulin')

In [ ]:
diabetes_data.loc[(diabetes_data ['Outcome'] == 0) & (diabetes_data['Insulin'].isnull()), 'Insulin'] = 102.5
diabetes_data.loc[(diabetes_data ['Outcome'] == 1) & (diabetes_data['Insulin'].isnull()), 'Insulin'] = 169.5

In [ ]:
median_target('Glucose')

In [ ]:
diabetes_data.loc[(diabetes_data ['Outcome'] == 0) & (diabetes_data['Glucose'].isnull()), 'Glucose'] = 107
diabetes_data.loc[(diabetes_data ['Outcome'] == 1) & (diabetes_data['Glucose'].isnull()), 'Glucose'] = 104

In [ ]:
median_target('SkinThickness')

In [ ]:
diabetes_data.loc[(diabetes_data ['Outcome'] == 0) & (diabetes_data['SkinThickness'].isnull()), 'SkinThickness'] = 27
diabetes_data.loc[(diabetes_data ['Outcome'] == 1) & (diabetes_data['SkinThickness'].isnull()), 'SkinThickness'] = 32

In [ ]:
diabetes_data.head(15)

The minimum values after imputation changed, there are no zero values anymore except for the pregnancies, as here it is hard to judge if the zero values are meaningful data or missing data.

In [ ]:
diabetes_data.describe()

As can be seen from the describe function of the data, the top quartile for the age starts from 41, the following therefore displays the mean blood pressure for this age category and the result is 78.

In [ ]:
TopQuartile_MeanBloodPressure = diabetes_data.loc[diabetes_data['Age']>= 41, 'BloodPressure'].mean()
print(TopQuartile_MeanBloodPressure)